In [19]:
import pandas as pd
import numpy as np
import json
import csv
import os
import re
from spacy.kb import KnowledgeBase
import spacy

from article_loader_parser import article_loader_to_df
from sentiment_analysis_functions import *
#from word_cleaning_functions import *
#from topic_modeling_functions import *
#from named_entity_recognition_articles import *




In [ ]:
global_path = 'C:/Users/17742/Desktop/win_art_writing/art_writing/text_cleaned'
path = 'text_cleaned_all'

folder_path = os.path.join(global_path, path)#global_path + path os.path.expanduser()

data = article_loader_to_df(folder_path=folder_path, iterable=100, israndom=True)
#data

In [ ]:
nlp = spacy.load('en_core_web_trf')  #lg')

In [ ]:
def spacy_importer_prepper(data):
    entities = []
    for i in range(data.shape[0]):
        ents_found = []
        # print(i)
        # index, article in data.iterrows()
        article = data.iloc[int(i)]
        # article = article[0]
        para = article["para"]
        meta_dict = dict([("unique_id", article["unique_id"]), ("title", article["title"]), ("author", article["author"]),
                          ("pubtime", article["pubtime"])])
        #print(meta_dict)
        try:
            doc = nlp(para)
            for ent in doc.ents:
                #ent_dict = dict([("entity", str(ent.text)), ("label", str(ent.label_))])
                text = ent.text
                label = ent.label_
                print((text, label), end=', ')
                ent_dict = {"txt": str(text), "label": str(text)}
                ents_found.append(ent_dict)
                #print(ent_dict)
        except TypeError:
            print(article['unique_id'])
            pass
        for found_ent in ents_found:
            all_dict = found_ent.update(meta_dict)
            #print(all_dict, end=', ')
            entities.append(all_dict)
        print("  \n")
    return entities

In [ ]:
data.set_index("unique_id")
data.dtypes


In [63]:
article = data.iloc[int(94)]
para = article["para"]
doc = nlp(para)

In [64]:
org_list = []
#help(doc)
for entity in doc.ents:
    print(f"{entity.label_} :  {entity.text}")
    if entity.label_ == 'ORG':
        org_list.append(entity.text)

PERSON :  Agnes de Mille
ORG :  New York Theater Ballet’s
WORK_OF_ART :  Dance/Speak: The Life of Agnes de Mille
DATE :  Friday
TIME :  night
FAC :  Gould Hall
PERSON :  Patricia Conolly
ORDINAL :  first
TIME :  three-hour
ORG :  de Mille’s
WORK_OF_ART :  Dance/Speak
PERSON :  Scott Alan Evans
PERSON :  Anderson Ferrell
DATE :  two years
PERSON :  de Mille
WORK_OF_ART :  Dance/Speak
PERSON :  Conolly
PERSON :  de Mille
DATE :  today
WORK_OF_ART :  Rodeo
EVENT :  Fall River Legend
WORK_OF_ART :  Three Virgins
WORK_OF_ART :  Dance/Speak
PERSON :  de Mille
PERSON :  Liza Gennaro
WORK_OF_ART :  One Touch of Venus
ORG :  Carousel
WORK_OF_ART :  Bloomer Girl
WORK_OF_ART :  Goldilocks
GPE :  Oklahoma
PERSON :  Jud
PERSON :  Edward Lawrence
PERSON :  Elena Zahlmann
WORK_OF_ART :  Rodeo
PERSON :  de Mille
WORK_OF_ART :  Dance/Speak
CARDINAL :  one


In [57]:
spacy.explain('NORP')


'Nationalities or religious or political groups'

In [65]:
org_list


['New York Theater Ballet’s', 'de Mille’s', 'Carousel']

In [ ]:
#plot = timeplot_sentiment(data, "polarity", path)
#from named_entity_recognition_articles import spacy_importer_prepper

In [ ]:
entities1 = spacy_importer_prepper(data)

In [ ]:
df = pd.DataFrame(entities1)
df

In [ ]:
artistfilt = df[df['label'] == "PERSON" | ['label'] == "WORK_OF_ART"]
artistfilt

In [ ]:
artworks = df[df['label'] == "WORK_OF_ART"]
artworks


In [30]:
def load_entities():
    entities_loc =  os.path.join('C:/Users/17742/Desktop/win_art_writing/art_galleries_nyc' , "ART_GALLERY.csv")
    names = dict()
    descriptions = dict()
    locations = dict()
    intid = 0
    with open(entities_loc, "r", encoding="utf8") as csvfile:
        csvreader = csv.DictReader(csvfile, delimiter=",")
        for row in csvreader:
            intid +=1
            #print(row['NAME'])
            name = row['NAME']
            school = ["school" , "college" ,"university"]
            if any(x in name.lower() for x in school):
                desc = "academic institution"
            else:
                desc = "art gallery"
            loc = row['CITY']
            alphanum = "A"
            qid = alphanum + str(intid)
            names[qid] = name
            descriptions[qid] = desc
            locations[qid] = loc
    return names, descriptions, locations

In [31]:
name_dict, desc_dict, loc_dict = load_entities()
for QID in name_dict.keys():
    print(f"{QID}, name= {name_dict[QID]}, desc= {desc_dict[QID]} located={loc_dict[QID]}")

A1, name= O'reilly William & Co Ltd, desc= art gallery located=New York
A2, name= Organization of Independent Artists - Gallery 402, desc= art gallery located=New York
A3, name= Owen Gallery, desc= art gallery located=New York
A4, name= P P O W Gallerie, desc= art gallery located=New York
A5, name= P P O W Inc, desc= art gallery located=New York
A6, name= P S 122 Gallery, desc= art gallery located=New York
A7, name= Pace Editions, desc= art gallery located=New York
A8, name= Pace MacGill Gallery, desc= art gallery located=New York
A9, name= Pace Prints & Primitive Gallery, desc= art gallery located=New York
A10, name= Pace Wildenstein, desc= art gallery located=New York
A11, name= Pace Wildenstein, desc= art gallery located=New York
A12, name= Paige's Art Gallery, desc= art gallery located=New York
A13, name= PaineWebber Gallery, desc= art gallery located=New York
A14, name= Paint Can Studios, desc= art gallery located=Astoria
A15, name= Paley Barbara K, desc= art gallery located=New Y

In [37]:
from spacy.kb import KnowledgeBase
kb = KnowledgeBase(vocab=nlp.vocab, entity_vector_length=300)

In [38]:
for qid, desc in desc_dict.items():
    desc_doc = nlp(desc)
    desc_enc = desc_doc.vector
    kb.add_entity(entity=qid, entity_vector=desc_enc, freq=342)

In [39]:
for qid, name in name_dict.items():
    kb.add_alias(alias=name, entities=[qid], probabilities=[1])   # 100% prior probability P(entity|alias)

<ipython-input-39-c4df2bdc9476>:2: UserWarning: [W017] Alias 'Pace Wildenstein' already exists in the Knowledge Base.
  kb.add_alias(alias=name, entities=[qid], probabilities=[1])   # 100% prior probability P(entity|alias)
<ipython-input-39-c4df2bdc9476>:2: UserWarning: [W017] Alias 'Plaza Holding Inc' already exists in the Knowledge Base.
  kb.add_alias(alias=name, entities=[qid], probabilities=[1])   # 100% prior probability P(entity|alias)
<ipython-input-39-c4df2bdc9476>:2: UserWarning: [W017] Alias 'Ceres Gallery' already exists in the Knowledge Base.
  kb.add_alias(alias=name, entities=[qid], probabilities=[1])   # 100% prior probability P(entity|alias)
<ipython-input-39-c4df2bdc9476>:2: UserWarning: [W017] Alias 'Guggenheim Asher Assocs Inc' already exists in the Knowledge Base.
  kb.add_alias(alias=name, entities=[qid], probabilities=[1])   # 100% prior probability P(entity|alias)
<ipython-input-39-c4df2bdc9476>:2: UserWarning: [W017] Alias 'Decor Art Gallery Inc' already exists

In [40]:
qids = name_dict.keys()
size = 1 / len(qids)
probs = [size for qid in qids]
kb.add_alias(alias="Emerson", entities=qids, probabilities=probs)  # sum([probs]) should be <= 1 !

4831166512461469197

In [41]:
print(f"Entities in the KB: {kb.get_entity_strings()}")
print(f"Aliases in the KB: {kb.get_alias_strings()}")

Entities in the KB: ['A7', 'A550', 'A640', 'A87', 'A639', 'A782', 'A82', 'A605', 'A642', 'A845', 'A193', 'A499', 'A208', 'A709', 'A36', 'A740', 'A15', 'A57', 'A250', 'A586', 'A658', 'A592', 'A116', 'A156', 'A249', 'A792', 'A209', 'A92', 'A521', 'A2', 'A593', 'A385', 'A465', 'A175', 'A281', 'A778', 'A871', 'A876', 'A909', 'A711', 'A569', 'A741', 'A486', 'A457', 'A787', 'A456', 'A424', 'A578', 'A655', 'A696', 'A446', 'A48', 'A246', 'A308', 'A856', 'A292', 'A55', 'A351', 'A562', 'A682', 'A783', 'A255', 'A377', 'A189', 'A604', 'A692', 'A223', 'A579', 'A686', 'A695', 'A468', 'A810', 'A143', 'A762', 'A356', 'A360', 'A493', 'A594', 'A35', 'A548', 'A270', 'A386', 'A744', 'A371', 'A581', 'A884', 'A625', 'A522', 'A661', 'A644', 'A13', 'A184', 'A352', 'A766', 'A169', 'A719', 'A530', 'A261', 'A384', 'A697', 'A767', 'A445', 'A508', 'A144', 'A675', 'A889', 'A167', 'A24', 'A279', 'A192', 'A12', 'A31', 'A724', 'A148', 'A790', 'A38', 'A506', 'A354', 'A544', 'A75', 'A286', 'A139', 'A539', 'A750', 'A805'

In [46]:
print(f"Candidates for 'Norman': {[c.entity_ for c in kb.get_alias_candidates('Norm')]}")
print(f"Candidates for 'Merrin': {[c.entity_ for c in kb.get_alias_candidates('Merrin')]}")
print(f"Candidates for 'Sofia': {[c.entity_ for c in kb.get_alias_candidates('Sofia')]}")

Candidates for 'Norman': []
Candidates for 'Merrin': []
Candidates for 'Sofia': []


In [47]:
import os
output_dir = Path.cwd().parent / "my_output"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
kb.dump(output_dir / "my_kb")
nlp.to_disk(output_dir / "my_nlp")

NameError: name 'Path' is not defined